# Efficient representation of multidimensional arrays.
### Last modification (05.06.2018)

![storage_complexity](./images/storage_complexity.png)


In this tutorial we provide a theoretical backgound on efficient representation of multidimensional arrays and show how these data structures are integrated into [hottbox](https://github.com/hottbox/hottbox) through **TensorCPD**, **TensorTKD** and **TensorTT** classes.

More details on **TensorCPD**, **TensorTKD** and **TensorTT** classes can be found on our [documentation page](https://hottbox.github.io/stable/api/hottbox.core.html#module-hottbox.core).

**Note:** this tutorial assumes that you are familiar with the basics of tensor algebra and the corresponding conventional notation.  If you are new to this area, the required background is covered in our [introductory notebook](https://github.com/hottbox/hottbox-tutorials/blob/master/1_N-dimensional_arrays_and_Tensor_class.ipynb).

**Requirements:** ``hottbox==0.1.3``

**Authors:** 
Ilya Kisil (ilyakisil@gmail.com); 
Giuseppe G. Calvi (ggc115@ic.ac.uk)

In [1]:
import numpy as np
from hottbox.core import Tensor, TensorCPD, TensorTKD, TensorTT

# Outer product, rank-1 tensor and definitions of rank of a multi-dimensional array.


The central operator in tensor analysis is the outer product (sometimes refered to as the tensor product). 
Consider tensors $\mathbf{\underline{A}} \in \mathbb{R}^{I_1 \times \cdots \times I_N}$ and $\mathbf{\underline{B}}  \in \mathbb{R}^{J_1 \times \cdots \times J_M}$, then  their outer product yeilds a tensor of higher order then both of them:

$$
\begin{equation}
\begin{aligned}
    \mathbf{\underline{A}} \circ \mathbf{\underline{B}} &= \mathbf{\underline{C}} \in \mathbb{R}^{I_1 \times \cdots \times I_N \times J_1 \times \cdots \times J_M} \\
    a_{i_1,\dots,i_N}b_{j_1,\dots,j_M} &= c_{i_1,\dots,i_N,j_1,\dots,j_M} 
\end{aligned}    
\end{equation}
$$

Most of the time we deal with the outer product of vectors, which significanlty simplifies the general form expressed above and establishes one the of the most fundamenatal definitions. A tensor of order $N$ is said to be of **rank-1** if it can be represented as an outer product of $N$ vectors. The figure below illustrates an example of rank-1 tensor $\mathbf{\underline{X}}$ and provides intuition of how operation of outer product is computed:

![outerproduct](./images/outerproduct_3.png)

There are several forms of the rank of N-dimensional arrays each of which is accosiated with a representation of a tensor in a particular form:

1. Kruskal rank $\rightarrow$ canonical polyadic form.

- Multi-linear rank $\rightarrow$ tucker form.

- TT rank $\rightarrow$ tensor train form.

Each of this representations has the correposing class: **``TensorCPD``**, **``TensorTKD``**, **``TensorTT``**. All of them come with almost identical API except for obejct creation and, as a result, the names for some attributes. But before, we can proceed, it is crucial to get acquainted with the following definitions.

# Canonical Polydiac representation (CP), Kruskal rank and TensorCPD class

![cpd_as_rank_one](./images/cpd_as_rank_one.png)

## Kryskal rank
This figure illustrates a tensor $\mathbf{\underline{X}}$ of rank $R$. The **rank** of a tensor $\mathbf{\underline{X}}$ is defined as the smallest number of rank-one tensors that produce $\mathbf{\underline{X}}$ as their linear combination. This definition of a tensor rank is also known as the **Kruskal rank**.

## CP representation
For a third order tensor or rank $R$ it can be expressed as follows:

$$\mathbf{\underline{X}} = \sum_{r=1}^R \mathbf{\underline{X}}_r = \sum_{r=1}^R \lambda_{r} \cdot \mathbf{a}_r \circ \mathbf{b}_r \circ \mathbf{c}_r$$

The vectors $\mathbf{a}_r, \mathbf{b}_r$ and $\mathbf{c}_r$ are oftentime combined into corresponding **factor matrices**:

$$
\mathbf{A} = \Big[ \mathbf{a}_1 \cdots \mathbf{a}_R \Big] \quad
\mathbf{B} = \Big[ \mathbf{b}_1 \cdots \mathbf{b}_R \Big] \quad
\mathbf{C} = \Big[ \mathbf{c}_1 \cdots \mathbf{c}_R \Big] \quad
$$

Thus, if we employ the mode-$n$ product, the canonical polyadic representation takes form:

$$
\mathbf{\underline{X}} = \mathbf{\underline{\Lambda}} \times_1 \mathbf{A} \times_2 \mathbf{B} \times_3 \mathbf{C} = \Big[\mathbf{\underline{\Lambda}}; \mathbf{A}, \mathbf{B}, \mathbf{C} \Big]
$$

where the elements on the super-diagonal of $\mathbf{\underline{\Lambda}}$ are occupied by the values $\lambda_r$ and all other equal to zero. This is the **canonical polyadic (CP)** representation of the original tensor
and can be visualised as shown on figure below:

![tensorcpd](./images/TensorCPD.png)


## TensorCPD class in hottbox

In **`hottbox`**, this form is available through the **``TensorCPD``** class. In order to create such object, you need to pass a list of factor matrices (2d numpy arrays) and a vector of values (as 1d numpy array) for the main diagonal:

```python
tensor_cpd = TensorCPD(fmat=[A, B, C], core_values=values)
```

**Note:** all matrices should have the same number of columns and be equal to the length of ``values``

In [2]:
I, J, K = 3, 4, 5  # define shape of the tensor in full form
R = 2              # define Kryskal rank of a tensor in CP form 

A = np.arange(I * R).reshape(I, R)
B = np.arange(J * R).reshape(J, R)
C = np.arange(K * R).reshape(K, R)
values = np.arange(R)

tensor_cpd = TensorCPD(fmat=[A, B, C], core_values=values)
print(tensor_cpd)

Kruskal representation of a tensor with rank=(2,).
Factor matrices represent properties: ['mode-0', 'mode-1', 'mode-2']
With corresponding latent components described by (3, 4, 5) features respectively.


The list of factor matrices **[A, B, C]** is stored in **`_fmat`** placeholder which can (should) be accessed through the correspodning property **`fmat`**. The values for the super-diagonal are stored in **`_core_values`** placeholder. But there is no direct access to them, because they are used fore creation of the core tensor:

```python
tensor_cpd.core
```

This returns an object of the **``Tensor``** class with the **``_core_values``** placed on its super-diagonal.

In [3]:
print('\tFactor matrices')
for mode, fmat in enumerate(tensor_cpd.fmat):
    print('Mode-{} factor matrix is of shape {}'.format(mode, fmat.shape))
    
print('\n\tCore tensor')
print(tensor_cpd.core)
tensor_cpd.core.data

	Factor matrices
Mode-0 factor matrix is of shape (3, 2)
Mode-1 factor matrix is of shape (4, 2)
Mode-2 factor matrix is of shape (5, 2)

	Core tensor
This tensor is of order 3 and consists of 8 elements.
Sizes and names of its modes are (2, 2, 2) and ['mode-0', 'mode-1', 'mode-2'] respectively.


array([[[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 1.]]])

In order to convert **``TensorCPD``** into the full representation, simply call: 

```python
tensor_cpd.reconstruct()
```

This returns an object of the **``Tensor``** class with N-dimensional array calculated as described above and being assinged to the **``_data``** attibute.

In [4]:
tensor_full = tensor_cpd.reconstruct()
print(tensor_full)
tensor_full.data

This tensor is of order 3 and consists of 60 elements.
Sizes and names of its modes are (3, 4, 5) and ['mode-0', 'mode-1', 'mode-2'] respectively.


array([[[  1.,   3.,   5.,   7.,   9.],
        [  3.,   9.,  15.,  21.,  27.],
        [  5.,  15.,  25.,  35.,  45.],
        [  7.,  21.,  35.,  49.,  63.]],

       [[  3.,   9.,  15.,  21.,  27.],
        [  9.,  27.,  45.,  63.,  81.],
        [ 15.,  45.,  75., 105., 135.],
        [ 21.,  63., 105., 147., 189.]],

       [[  5.,  15.,  25.,  35.,  45.],
        [ 15.,  45.,  75., 105., 135.],
        [ 25.,  75., 125., 175., 225.],
        [ 35., 105., 175., 245., 315.]]])

# Tucker representation, Multi-linear rank and TensorTKD class

## Multi-linear rank

The **multi-linear rank** of a tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I_1 \times \cdots \times I_N}$ is the $N$-tuple $(R_1, \dots, R_N)$ where each $R_n$ is the rank of the subspace spanned by mode-$n$ fibers, i.e. $R_n = \text{rank} \big( \mathbf{X}_{(n)} \big)$. For a tensor of order $N$ the values $R_1, R_2, \dots , R_N$ are not necessarily the same, whereas, for matrices (tensors of order 2) the equality $R_1 = R_2$ always holds, where $R_1$ and $R_2$ are the matrix column rank and row rank respectively.


## Tucker representation
![tensortkd](./images/TensorTKD.png)

For a tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I \times J \times K}$ illustrated above, the **tucker form** represents it as a dense core tensor $\mathbf{\underline{G}}$ with multi-linear rank ($Q, R, P$) and a set of factor matrices $\mathbf{A} \in \mathbb{R}^{I \times Q}, \mathbf{B} \in \mathbb{R}^{J \times R}$ and $\mathbf{C} \in \mathbb{R}^{K \times P}$.

The tucker form of a tensor is closely related to the CP form and can be expressed through a 
sequence of mode-$n$ products in a similar way.

$$
\mathbf{\underline{X}} = \mathbf{\underline{G}} \times_1 \mathbf{A} \times_2 \mathbf{B} \times_3 \mathbf{C} = \Big[\mathbf{\underline{G}}; \mathbf{A}, \mathbf{B}, \mathbf{C} \Big]
$$

## TensorTKD class in hottbox

In **`hottbox`**, this form is available through the **``TensorTKD``** class. In order to create such object, you need to pass a list of $N$ factor matrices (2d numpy arrays) and values for the core tensor (as n-dimensional numpy array):

```python
tensor_tkd = TensorTKD(fmat=[A, B, C], core_values=values)
```

**Note:** the number of columns in each of the factor matrices should be the same as the corresponding size of the numpy array with the values for the core tensor

In [5]:
I, J, K = 5, 6, 7  # define shape of the tensor in full form
Q, R, P = 2, 3, 4  # define multi-linear rank of the tensor in Tucker form

A = np.arange(I * Q).reshape(I, Q)
B = np.arange(J * R).reshape(J, R)
C = np.arange(K * P).reshape(K, P)
values = np.arange(Q * R * P).reshape(Q, R, P)

tensor_tkd = TensorTKD(fmat=[A, B, C], core_values=values)
print(tensor_tkd)

Tucker representation of a tensor with multi-linear rank=(2, 3, 4).
Factor matrices represent properties: ['mode-0', 'mode-1', 'mode-2']
With corresponding latent components described by (5, 6, 7) features respectively.


By analogy with the **`TensorCPD`**, the list of factor matrices **[A, B, C]** is stored in **`_fmat`** placeholder which can (should) be accessed through the correspodning property **`fmat`**. Similarly, the values of the core tensor are stored in **`_core_values`** placeholder and they cannot (should not) be accessed directly, because they are used to create a core tensors as an object of **`Tensor`** class, when the corresponding property is called:

```python
tensor_tkd.core
```

**Note:** the core values occupy all data values of a core tensor, as opposed to **`TensorCPD`** class where they are placed on the main diagonal.

In [6]:
print('\tFactor matrices')
for mode, fmat in enumerate(tensor_tkd.fmat):
    print('Mode-{} factor matrix is of shape {}'.format(mode, fmat.shape))
    
print('\n\tCore tensor')
print(tensor_tkd.core)
tensor_tkd.core.data

	Factor matrices
Mode-0 factor matrix is of shape (5, 2)
Mode-1 factor matrix is of shape (6, 3)
Mode-2 factor matrix is of shape (7, 4)

	Core tensor
This tensor is of order 3 and consists of 24 elements.
Sizes and names of its modes are (2, 3, 4) and ['mode-0', 'mode-1', 'mode-2'] respectively.


array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In order to convert **``TensorTKD``** into the full representation, simply call: 

```python
tensor_tkd.reconstruct()
```

This return an object of the **``Tensor``** class with N-dimensional array calculated as 
described above and being assinged to the **``_data``** attibute.

In [7]:
tensor_full = tensor_tkd.reconstruct()
print(tensor_full)
tensor_full.data

This tensor is of order 3 and consists of 210 elements.
Sizes and names of its modes are (5, 6, 7) and ['mode-0', 'mode-1', 'mode-2'] respectively.


array([[[    378,    1346,    2314,    3282,    4250,    5218,    6186],
        [   1368,    4856,    8344,   11832,   15320,   18808,   22296],
        [   2358,    8366,   14374,   20382,   26390,   32398,   38406],
        [   3348,   11876,   20404,   28932,   37460,   45988,   54516],
        [   4338,   15386,   26434,   37482,   48530,   59578,   70626],
        [   5328,   18896,   32464,   46032,   59600,   73168,   86736]],

       [[   1458,    5146,    8834,   12522,   16210,   19898,   23586],
        [   5112,   17944,   30776,   43608,   56440,   69272,   82104],
        [   8766,   30742,   52718,   74694,   96670,  118646,  140622],
        [  12420,   43540,   74660,  105780,  136900,  168020,  199140],
        [  16074,   56338,   96602,  136866,  177130,  217394,  257658],
        [  19728,   69136,  118544,  167952,  217360,  266768,  316176]],

       [[   2538,    8946,   15354,   21762,   28170,   34578,   40986],
        [   8856,   31032,   53208,   75384,   

# Tensor Train representation, TT-rank and TensorTT class

## Tensor Train representation

![tensortt](./images/TensorTT.png)

**Tensor trains (TTs)** are the simplest kinds of tensor networks, i.e. a decomposition of a high-order tensor in a set of sparsely interconnected lower-order tensors and factor matrices. Mathematically, an $N$-th order tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I_1 \times I_2 \times \cdots \times I_N}$ can be expressed as a TT as

$$
\mathbf{\underline{X}} = \mathbf{A} \times^1_2 \mathbf{\underline{G}}^{(1)}  \times^1_3 \mathbf{\underline{G}}^{(2)}   \times^1_3 \cdots \times^1_3 \mathbf{\underline{G}}^{(N-1)} \times^1_3 \mathbf{B} = \Big[  \mathbf{A}, \mathbf{\underline{G}}^{(1)}, \mathbf{\underline{G}}^{(2)}, \cdots, \mathbf{\underline{G}}^{(N-1)}, \mathbf{B}  \Big]
$$

Each element of a TT is generally referred to as **TT-core**, and $\mathbf{A} \in \mathbb{R}^{I_1 \times R_1}$, $\mathbf{B} \in \mathbb{R}^{R_{N-1}\times I_N}$, $\mathbf{\underline{G}}^{(n)} \in \mathbb{R}^{R_n \times I_{n+1} \times R_{n+1}}$ and the tuple $(R_1, R_2, \dots, R_{N-1})$ is called the **TT-rank**.


## TensorTT class in hottbox

In **`hottbox`**, this form is available through the **``TensorTT``** class. In order to create such object, you need to pass a list of values (as numpy arrays) for 
cores:

```python
tensor_tt = TensorTT(core_values=values)
```

In [8]:
I, J, K = 4, 5, 6  # define shape of the tensor in full form
R1, R2 = 2, 3      # define tt rank of the tensor in Tensor train form

values_1 = np.arange(I * R1).reshape(I, R1)
values_2 = np.arange(R1 * J * R2).reshape(R1, J, R2)
values_3 = np.arange(R2 * K).reshape(R2, K)

tensor_tt = TensorTT(core_values=[values_1, values_2, values_3])
print(tensor_tt)

Tensor train representation of a tensor with tt-rank=(2, 3).
Shape of this representation in the full format is (4, 5, 6).
Physical modes of its cores represent properties: ['mode-0', 'mode-1', 'mode-2']


The list of values for these core tensors is stored in **`_core_values`** placeholder. They should not be accessed directly, because they are used
for creation of **`Tensor`** class objects each of which represent a particular tt-core. The list of all cores can be accessed as 

```python
tensor_tt.cores
```

**Note:** All components of the Tensor Train representation are conventionally considered to be a core therefore, even matrices are objects of **`Tensor`** class.

In [9]:
for i, tt_core in enumerate(tensor_tt.cores):        
    print('\n\tCore tensor #{} of TT representation'.format(i))    
    print(tt_core)    
    print(tt_core.data)


	Core tensor #0 of TT representation
This tensor is of order 2 and consists of 8 elements.
Sizes and names of its modes are (4, 2) and ['mode-0', 'mode-1'] respectively.
[[0 1]
 [2 3]
 [4 5]
 [6 7]]

	Core tensor #1 of TT representation
This tensor is of order 3 and consists of 30 elements.
Sizes and names of its modes are (2, 5, 3) and ['mode-0', 'mode-1', 'mode-2'] respectively.
[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]
  [ 9 10 11]
  [12 13 14]]

 [[15 16 17]
  [18 19 20]
  [21 22 23]
  [24 25 26]
  [27 28 29]]]

	Core tensor #2 of TT representation
This tensor is of order 2 and consists of 18 elements.
Sizes and names of its modes are (3, 6) and ['mode-0', 'mode-1'] respectively.
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]]


If you what to access a specific tt-core of the TT representation, then it is more efficient to use a corresponding method which takes a positional number of desired core as input parameters

```python
tensor_tt.core(i=0)
```

**Note:** this parameter should not exceed the order of TT representation

In [10]:
for i in range(tensor_tt.order):
    tt_core = tensor_tt.core(i)
    print('\n\tCore tensor #{} of TT representation'.format(i))    
    print(tt_core)    
    print(tt_core.data)


	Core tensor #0 of TT representation
This tensor is of order 2 and consists of 8 elements.
Sizes and names of its modes are (4, 2) and ['mode-0', 'mode-1'] respectively.
[[0 1]
 [2 3]
 [4 5]
 [6 7]]

	Core tensor #1 of TT representation
This tensor is of order 3 and consists of 30 elements.
Sizes and names of its modes are (2, 5, 3) and ['mode-0', 'mode-1', 'mode-2'] respectively.
[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]
  [ 9 10 11]
  [12 13 14]]

 [[15 16 17]
  [18 19 20]
  [21 22 23]
  [24 25 26]
  [27 28 29]]]

	Core tensor #2 of TT representation
This tensor is of order 2 and consists of 18 elements.
Sizes and names of its modes are (3, 6) and ['mode-0', 'mode-1'] respectively.
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]]


In order to convert **``TensorTT``** into the full representation, simply call: 

```python
tensor_tt.reconstruct()
```

This return an object of the **``Tensor``** class with N-dimensional array calculated as described above and being assinged to the **``_data``** attibute.

In [11]:
tensor_full = tensor_tt.reconstruct()
print(tensor_full)
tensor_full.data

This tensor is of order 3 and consists of 120 elements.
Sizes and names of its modes are (4, 5, 6) and ['mode-0', 'mode-1', 'mode-2'] respectively.


array([[[ 300,  348,  396,  444,  492,  540],
        [ 354,  411,  468,  525,  582,  639],
        [ 408,  474,  540,  606,  672,  738],
        [ 462,  537,  612,  687,  762,  837],
        [ 516,  600,  684,  768,  852,  936]],

       [[ 960, 1110, 1260, 1410, 1560, 1710],
        [1230, 1425, 1620, 1815, 2010, 2205],
        [1500, 1740, 1980, 2220, 2460, 2700],
        [1770, 2055, 2340, 2625, 2910, 3195],
        [2040, 2370, 2700, 3030, 3360, 3690]],

       [[1620, 1872, 2124, 2376, 2628, 2880],
        [2106, 2439, 2772, 3105, 3438, 3771],
        [2592, 3006, 3420, 3834, 4248, 4662],
        [3078, 3573, 4068, 4563, 5058, 5553],
        [3564, 4140, 4716, 5292, 5868, 6444]],

       [[2280, 2634, 2988, 3342, 3696, 4050],
        [2982, 3453, 3924, 4395, 4866, 5337],
        [3684, 4272, 4860, 5448, 6036, 6624],
        [4386, 5091, 5796, 6501, 7206, 7911],
        [5088, 5910, 6732, 7554, 8376, 9198]]])

# Further reading list
- Tamara G. Kolda and Brett W. Bader, "Tensor decompositions and applications." SIAM REVIEW, 51(3):455–500, 2009.

- Ivan V. Oseledets,  "Tensor-train decomposition." SIAM Journal on Scientific Computing 33.5 (2011): 2295-2317.